#  Практика.


In [1]:
import numpy as np
import pandas as pd
import scipy.stats as ss
from typing import Callable
import pandas as pd

In [2]:
from scipy.optimize import minimize, LinearConstraint, NonlinearConstraint

In [3]:
df = pd.read_csv('hw_6_data.txt', sep=' ', names=['1', '2', '3', 'winning', 'reporter?'])
df.head()

,1,2,3,winning,reporter?
0,2,0,0,0,1
1,0,5,1,0,1
2,0,0,0,0,1
3,2,0,0,0,1
4,0,0,0,0,1


In [4]:
df = df[df['reporter?'] == 1]

In [5]:
df['reporter?'].mean()

1.0

In [6]:
df = df - (df > 4)
df.head()

,1,2,3,winning,reporter?
0,2,0,0,0,1
1,0,4,1,0,1
2,0,0,0,0,1
3,2,0,0,0,1
4,0,0,0,0,1


In [7]:
def correction(df, name):
    ret = np.unique(df[name], return_counts = True)[1]
    if name == '2': 
        ret = np.append(ret, ret[5])
        ret[5] = 0
    return ret / 138

In [8]:
names = ['1', '2', '3']
model = np.vstack([correction(df, name) for name in names])
model

array([[0.42753623, 0.35507246, 0.10144928, 0.04347826, 0.02173913,
        0.00724638, 0.04347826],
       [0.61594203, 0.05797101, 0.17391304, 0.11594203, 0.00724638,
        0.        , 0.02898551],
       [0.55797101, 0.2826087 , 0.04347826, 0.00724638, 0.03623188,
        0.02173913, 0.05072464]])

In [9]:
def likehood(theta: pd.DataFrame, model) -> float:
    return -np.sum(np.log(theta) * model)

In [10]:
expected = np.array([1 / 2, 1 / 4, 1 / 8, 1 / 16, 1 /64, 1 / 64, 1 / 32])
theta = np.array([expected,
                 expected,
                 expected])
theta

array([[0.5     , 0.25    , 0.125   , 0.0625  , 0.015625, 0.015625,
        0.03125 ],
       [0.5     , 0.25    , 0.125   , 0.0625  , 0.015625, 0.015625,
        0.03125 ],
       [0.5     , 0.25    , 0.125   , 0.0625  , 0.015625, 0.015625,
        0.03125 ]])

##  Ограничения.

 * $\theta \geq 0$
 .$0 \leq I_{21} \dot \theta \leq \text{np.inf}$
 * $\sum_i p_i^j = 1$.
 1 $\leq (111100000000\\
                               000011110000\\
                               000000001111)\theta \leq 1 $
 * $\mathbb E \text{reward} $.
 $\mathbb E = p^3 \otimes (p^2 \otimes (p^1 \otimes M))$
 
 Надо еще найти $\theta_0$, чутка уменьшить вероятность 0 и увеличить вероянтость алмазика, пока $\mathbb E \ne 0.92$

# Домашка.

## 1.

In [11]:
brain = [1380, 1344, 1356, 1291, 1308, 1271, 1371, 1430,
         1381, 1457, 1492, 1240, 1256, 1466, 1214, 1448,
         1510, 1395, 1507, 1264, 1293, 1251, 1380, 1386,
         1411, 1434, 1302, 1529, 1352, 1494, 1348, 1464,
         1286, 1345, 1491, 1259, 1541, 1214, 1310, 1286]

In [12]:
ss.norm.cdf(np.mean(brain), loc = 1400, scale = np.std(brain, ddof=1) / np.sqrt(len(brain)))

0.018568470027286878

In [13]:
np.mean(brain)

1368.925

In [14]:
ss.ttest_1samp(brain, 1400, alternative='less')

Ttest_1sampResult(statistic=-2.0842554201754617, pvalue=0.02186741366047351)

In [15]:
t = np.sqrt(len(brain)) * (np.mean(brain) - 1400) / np.std(brain, ddof=1)

ss.t.cdf(t, df=len(brain) - 1)

0.02186741366047351

## 2.

In [16]:
n = len(brain)

In [17]:
def likehood(P):
    return -sum(np.log(P))

In [18]:
def jac(P):
    return -1/P

In [19]:
def hes(P):
    return np.diag(1 / P ** 2)

In [20]:
ret = minimize(likehood, np.ones(n) / n, jac=jac, hess=hes,
              constraints = [
                  LinearConstraint(np.ones(n), 1, 1),
                  LinearConstraint(np.array(brain), 1400, 1400),
                  LinearConstraint(np.diag(np.ones(n)), 0, np.inf)
              ])

ret

D:\soft\Anaconda\lib\site-packages\scipy\optimize\_minimize.py:527: RuntimeWarning: Method SLSQP does not use Hessian information (hess).
  warn('Method %s does not use Hessian information (hess).' % method,
<ipython-input-17-d1f01677a211>:2: RuntimeWarning: invalid value encountered in log
  return -sum(np.log(P))
<ipython-input-17-d1f01677a211>:2: RuntimeWarning: divide by zero encountered in log
  return -sum(np.log(P))


     fun: 149.72983157007184
     jac: array([-42.77885952, -47.78752926, -46.1068651 , -55.15397095,
       -52.78636197, -57.93754352, -44.02829672, -35.8307194 ,
       -42.63959906, -32.07801178, -27.21085994, -62.24299761,
       -60.02146921, -30.82643658, -65.85397072, -33.32871043,
       -24.70818682, -40.69224578, -25.1256556 , -58.91180514,
       -54.88286823, -60.71658806, -42.7796677 , -41.94182102,
       -38.47161986, -35.27570894, -53.62124134, -22.0650749 ,
       -46.6776167 , -26.93183582, -47.22871622, -31.10133366,
       -55.84861417, -47.6495881 , -27.35034634, -59.60713413,
       -20.39516199, -65.85548493, -52.50789803, -55.84762883])
 message: 'Optimization terminated successfully'
    nfev: 158
     nit: 48
    njev: 48
  status: 0
 success: True
       x: array([0.02337603, 0.02092596, 0.02168874, 0.01813106, 0.01894429,
       0.01725997, 0.02271267, 0.02790901, 0.02345238, 0.031174  ,
       0.03675003, 0.01606606, 0.01666071, 0.03243969, 0.01518511,
   

In [21]:
boot = ss.rv_discrete(values=(brain, ret.x))

In [22]:
BIG = 1000000

sample = boot.rvs(size=(BIG, n), random_state=38)

In [23]:
sample = sample.mean(axis=1)
sample.size

1000000

In [24]:
np.sum(sample < np.mean(brain)) / BIG

0.01961

## 3.


In [25]:
n, m = 13, 37

BIG = 10000
X = ss.norm.rvs(size=(BIG, n))
Y = ss.norm.rvs(size=(BIG, m))

In [26]:
results = []

for _ in range(BIG):
    res = 0
    for x in X[_]:
        for y in Y[_]:
            res += x < y
    results.append(res)
    


In [27]:
np.quantile(results, 0.95)

316.0

## 4.


In [28]:
from itertools import product

In [29]:
df = pd.read_csv('hw_6_data.txt', sep=' ', names=['1', '2', '3', 'winning', 'reporter?'])
df.head()

,1,2,3,winning,reporter?
0,2,0,0,0,1
1,0,5,1,0,1
2,0,0,0,0,1
3,2,0,0,0,1
4,0,0,0,0,1


In [30]:
df = df[df['reporter?'] == 1]

In [31]:
def reward(x):
    a, b, c = x
    a -= a > 4
    b -= b > 4
    c -= c > 4
    cs = [0, 0, 0, 0, 0, 0, 0]
    cs[a] += 1
    cs[b] += 1
    cs[c] += 1
    trips = [0, 10, 25, 40, 800, 10, 80]
    for i, t in enumerate(cs):
        if t == 3: return trips[i]
    if sum(cs[1:4]) == 3: return 5
    if cs[5] == 2 and cs[4] == 0: return 5
    if cs[5] == 1 and cs[4] == 0: return 2
    if cs[4] == 1:
        for i, t in enumerate(cs):
            if t == 2: return 2 * trips[i]
        if cs[5] == 1: return 10
        if sum(cs[1:4]) == 2: return 10
    if cs[4] == 2:
        for i, t in enumerate(cs):
            if t == 1: return 4 * trips[i]
    return 0

In [32]:
cols = ['1', '2', '3']
probs = [np.unique(df[col], return_counts = True)[1] / df.shape[0] for col in cols]
probs[1] = np.append(probs[1], probs[1][-1])
probs[1][5] = 0.0
probs

[array([0.42753623, 0.35507246, 0.10144928, 0.04347826, 0.02173913,
        0.00724638, 0.04347826]),
 array([0.61594203, 0.05797101, 0.17391304, 0.11594203, 0.00724638,
        0.        , 0.02898551]),
 array([0.55797101, 0.2826087 , 0.04347826, 0.00724638, 0.03623188,
        0.02173913, 0.05072464])]

In [33]:
def mean_hypothesis(sample: pd.DataFrame):
    cnts = [[], [], []]
    for i, col in enumerate(['1', '2', '3']):
        res = []
        for val in [0, 1, 2, 3, 5, 6, 7]:
            cnts[i].append((sample[col] == val).sum())
    cnts_r = np.hstack(cnts)

    def obj_fun(x):
        return -1 * (np.log(x) * cnts_r).sum()
    
    def obj_grad(x):
        return -1 * cnts_r / x
    
    # функция reward(x: tuple[int, int, int]) -> int возвращает выигрыш соответствующей комбинации
    M = np.array([reward(x) for x in product([0, 1, 2, 3, 5, 6, 7], repeat=3)]).reshape((7, 7, 7))
    
    def constr_fun(x):
        p1, p2, p3 = x[0:7], x[7:14], x[14:21]
        return np.tensordot(p3, np.tensordot(p2, np.tensordot(p1, M, (0, 0)), (0, 0)), (0, 0))
    
    def constr_grad(x):
        p1, p2, p3 = x[0:7], x[7:14], x[14:21]
        p1_grad = np.tensordot(p3, np.tensordot(p2, M, (0, 1)), (0, 1))
        p2_grad = np.tensordot(p3, np.tensordot(p1, M, (0, 0)), (0, 1))
        p3_grad = np.tensordot(p2, np.tensordot(p1, M, (0, 0)), (0, 0))
        return np.hstack([p1_grad, p2_grad, p3_grad])
    
    lin_constr_1 = LinearConstraint(np.eye(21), 0.0001, 1)
    lin_constr_2 = LinearConstraint([[1] * 7 + [0] * 14, [0] * 7 + [1] * 7 + [0] * 7, [0] * 14 + [1] * 7], [1, 1, 1], [1, 1, 1])
    non_lin_constr = NonlinearConstraint(constr_fun, 0.9199, np.inf, jac=constr_grad)
    magic = 0.03839888129674522
    x0 = np.hstack([probs[0] + magic * np.array([-1, 0, 0, 0, 1, 0, 0]),
                    probs[1] + magic * np.array([-1, 0, 0, 0, 0, 1, 0]),
                    probs[2]])
    
    return minimize(obj_fun, x0=x0, jac=obj_grad, constraints=[lin_constr_1, lin_constr_2, non_lin_constr])

In [34]:
df.head()

,1,2,3,winning,reporter?
0,2,0,0,0,1
1,0,5,1,0,1
2,0,0,0,0,1
3,2,0,0,0,1
4,0,0,0,0,1


In [35]:
ret = mean_hypothesis(df)

In [36]:
theory = ret.x
observed = np.hstack(probs)

In [44]:
ss.chisquare(observed * 138, f_exp=theory * 138, ddof=17)

Power_divergenceResult(statistic=2.667007365139277, pvalue=0.4458631948171974)

In [39]:
theory

array([4.05696990e-01, 3.59810197e-01, 1.05956707e-01, 4.52682438e-02,
       3.20669781e-02, 8.10912352e-03, 4.30917612e-02, 5.85231885e-01,
       6.30304192e-02, 1.85099981e-01, 1.22734519e-01, 1.38905458e-02,
       1.00000174e-04, 2.99126505e-02, 5.28427762e-01, 2.90463981e-01,
       4.62477188e-02, 7.63268217e-03, 5.38145059e-02, 2.39371772e-02,
       4.94761730e-02])

## 5.

In [69]:
vals = [0, 1, 2, 3, 5, 6, 7]

In [70]:
col_1 = ss.rv_discrete(values=(vals, theory[:7]))
col_2 = ss.rv_discrete(values=(vals, theory[7:14]))
col_3 = ss.rv_discrete(values=(vals, theory[14:]))

In [135]:
BIG = 10000
sample = np.array([
    col_1.rvs(size=(138, BIG), random_state=38),
    col_2.rvs(size=(138, BIG), random_state=38),
    col_3.rvs(size=(138, BIG), random_state=38),
]).transpose()

In [136]:
sample.shape

(10000, 138, 3)

In [146]:
def E(arr):
    arr = arr.reshape(138, 3)
    cols = ['1', '2', '3']
    d = pd.DataFrame(arr, columns=cols)
    
    ps = mean_hypothesis(d).x
    
    M = np.array([reward(x) for x in product([0, 1, 2, 3, 5, 6, 7], repeat=3)]).reshape((7, 7, 7))
    return np.tensordot(ps[14:], np.tensordot(ps[7:14], np.tensordot(ps[:7], M, (0, 0)), (0, 0)), (0, 0))

In [147]:
Es = np.apply_along_axis(E, axis=1, arr=sample.reshape(BIG, 138 * 3))
Es.size

10000

In [148]:
def count_p(x, p=theory):
    a, b, c = x
    a -= a > 4
    b -= b > 4
    c -= c > 4
    return p[a] * p[b+7] * p[c+14]

In [149]:
cool_model = ss.rv_discrete(values=(np.array([reward(x) for x in product([0, 1, 2, 3, 5, 6, 7], repeat=3)]),
                                   np.array([count_p(x) for x in product([0, 1, 2, 3, 5, 6, 7], repeat=3)])))

In [150]:
Es.std() * np.sqrt(138) / cool_model.std()

0.31352044351362357